## Data Acquisition

In [1]:
import pandas as pd

In [2]:
btc_trades_df = pd.read_parquet('../datasets/btc_trades_df.parquet', engine='fastparquet')

In [3]:
# timestamp to python datetime
btc_trades_df['date'] = pd.to_datetime(btc_trades_df['date'], unit='s')

In [4]:
btc_trades_df = btc_trades_df.set_index(['date'])

In [5]:
btc_trades_df.head(10)

,type,price,amount
date,,,
2023-09-29 17:03:28,buy,135078.450000,0.015500
2023-09-29 17:03:28,buy,135078.450000,0.044400
2023-09-29 17:03:28,buy,135083.000000,0.159676
2023-09-29 17:03:32,buy,135083.000000,0.068600
2023-09-29 17:04:08,buy,135187.000000,0.000148
2023-09-29 17:04:18,sell,135187.000000,0.005700
2023-09-29 17:04:18,sell,135184.411374,0.012500
2023-09-29 17:04:18,sell,135150.627619,0.030200
2023-09-29 17:04:18,sell,135083.040000,0.018883


## Exploratory Data Analysis

In [6]:
# TBD

## Isolation Forest

In [7]:
import numpy as np
from sklearn.ensemble import IsolationForest

In [17]:
model = IsolationForest(n_estimators=1_000, max_samples='auto', contamination=float(0.1), max_features=1.0)

In [18]:
df = btc_trades_df.loc['2023-09-29':'2023-09-29'][['price', 'amount']]

In [19]:
df

,price,amount
date,,
2023-09-29 17:03:28,135078.450000,0.015500
2023-09-29 17:03:28,135078.450000,0.044400
2023-09-29 17:03:28,135083.000000,0.159676
2023-09-29 17:03:32,135083.000000,0.068600
2023-09-29 17:04:08,135187.000000,0.000148
...,...,...
2023-09-29 23:53:43,135573.140000,0.000221
2023-09-29 23:55:02,135479.486492,0.000023
2023-09-29 23:57:12,135479.486492,0.000015


In [20]:
model.fit(df[['price', 'amount']])

IsolationForest(contamination=0.1, n_estimators=1000)

In [21]:
df['scores'] = model.decision_function(df[['price', 'amount']])
df['anomaly'] = model.predict(df[['price', 'amount']])

In [22]:
model.estimators_[:9]

[ExtraTreeRegressor(max_depth=8, max_features=1, random_state=2040945102),
 ExtraTreeRegressor(max_depth=8, max_features=1, random_state=1813387650),
 ExtraTreeRegressor(max_depth=8, max_features=1, random_state=5646730),
 ExtraTreeRegressor(max_depth=8, max_features=1, random_state=373443867),
 ExtraTreeRegressor(max_depth=8, max_features=1, random_state=1276138775),
 ExtraTreeRegressor(max_depth=8, max_features=1, random_state=1548508640),
 ExtraTreeRegressor(max_depth=8, max_features=1, random_state=1102449817),
 ExtraTreeRegressor(max_depth=8, max_features=1, random_state=358714395),
 ExtraTreeRegressor(max_depth=8, max_features=1, random_state=901514532)]

In [23]:
scores = model.score_samples(df[['price', 'amount']])

In [24]:
np.mean(scores)

-0.4387105405497376

In [25]:
anomaly = df.loc[df['anomaly'] == -1]
anomaly_index = list(anomaly.index)
print(anomaly)

                             price    amount    scores  anomaly
date                                                           
2023-09-29 17:03:28  135078.450000  0.044400 -0.023472       -1
2023-09-29 17:03:28  135083.000000  0.159676 -0.131160       -1
2023-09-29 17:03:32  135083.000000  0.068600 -0.048483       -1
2023-09-29 17:04:18  135150.627619  0.030200 -0.008379       -1
2023-09-29 17:30:41  134925.410000  0.082284 -0.082074       -1
...                            ...       ...       ...      ...
2023-09-29 21:35:06  136089.610000  0.002172 -0.036907       -1
2023-09-29 22:14:21  135669.000000  0.092699 -0.019743       -1
2023-09-29 23:04:02  135811.920000  0.097879 -0.055349       -1
2023-09-29 23:04:02  135815.250000  0.067767 -0.033641       -1
2023-09-29 23:35:33  135429.040000  0.037892 -0.014786       -1

[75 rows x 4 columns]
